In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# classify process

In [ ]:
# data = pd.read_csv("data.csv")
# # data = data.sample(frac=1).reset_index(drop=True)
# data = data.drop(['task', 'CreationTimestamp', 'ProcessID', 'ParentPID', 'CommandLine', 'Image'] , axis = 1)
# cols = data.columns.values.tolist()

In [ ]:
# y = data['label']
# X = data.drop(['label'], axis = 1)
# X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [ ]:
# model = RandomForestClassifier(n_estimators=10)
# model.fit(X_train,y_train)
# y_pred = model.predict(X_test)

# print(classification_report(y_test,y_pred))
# print(accuracy_score(y_test, y_pred))
# print(time.time() - start)

# classify task 

In [2]:
import os
import pandas as pd
def getListTask(folder_name, label):
  df = pd.DataFrame() 
  for root, dir, file in os.walk(folder_name):
    for f in file:
      if '.csv' in f:
        print(os.path.join(root, f))
        df = pd.concat([df, pd.read_csv(os.path.join(root, f))], sort=False)
    # df = df.loc[df["label"] != 0]
    # df = df.dropna()
    data = list(set(df["task"].to_list())) 
    labels = [label for i in range(len(data))]
    return df, pd.DataFrame({"task":data, "label": labels}, columns= ["task", "label"])   
  

In [3]:
process_data = pd.DataFrame() 
task_data = pd.DataFrame() 
label = {"nor": 1,"mal" : 3, "susp" : 2 }
for root, dir, file in os.walk("/content/drive/My Drive/Data_2021/Data_9-2-2021/data"):
  for d in dir:
    # print(os.path.join(root, d))
    df_process, df_task =  getListTask(os.path.join(root, d),label[d] )
    process_data = pd.concat([process_data, df_process], sort=False)
    task_data = pd.concat([task_data, df_task], sort=True)
print(process_data.shape[0], task_data.shape[0])


/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_1.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_2.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_3.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_4.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_5.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_Minh.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/susp/Suci.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/nor/Normal_Hoa.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/nor/Normal_Hu.csv
820455 50985


In [ ]:
# process_data.isnull().sum()

In [7]:
# process_data = process_data.drop(["Unnamed: 0"], axis=1)
process_data.sample(5)
# task_data.sample(5)

,task,CreationTimestamp,ProcessID,ParentPID,CommandLine,Image,ProcessType,Registry,Files,Modules,DroppedFiles,DebugStrings,EventsCounters_Network,Scores_Network,Autostart,LowAccess,FileType,Priority,unknown_ip,Suspicious_ip,Malicious_ip,whitelist_ip,unsafe_ip,unknown_domain,Suspicious_domain,Malicious_domain,whitelist_domain,unsafe_domain,unknown_http_request,Suspicious_http_request,Malicious_http_request,whitelist_http_request,unsafe_http_request,HKEY_LOCAL_MACHINE,HKEY_CLASSES_ROOT,HKEY_USERS,HKEY_CURRENT_CONFIG,HKEY_CURRENT_USER,startup_registry_keys,active_setup_registry_keys,services_registry_keys,dll_injection_registry_keys,shell_spawning_registry_keys,internet_settings_registry_keys,bho_registry_keys,Unnamed: 45,label
76539,570732d9-8921-4b2a-892a-89df954fb270,1597735289480,2752,3264,"""C:\ProgramData\wndll.exe""",C:\ProgramData\wndll.exe,Child process,6,134,61,0,0,0,0,0,0,NKN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,NaN,3
35675,a0f4988c-8812-4d16-aa0e-bef1cc3e2616,1611354863406,2544,356,"""C:\Program Files\Adobe\Acrobat Reader DC\Read...",C:\Program Files\Adobe\Acrobat Reader DC\Reade...,Main process,96,11770,192,8,0,25,1,0,0,ADB,0,0,0,0,2,2,0,0,0,2,2,0,0,0,1,5,0,10,0,0,8,0,0,0,0,0,1,0,NaN,1
51348,27f2ba89-4c04-4a9e-9c61-a4b10370292f,1584090081698,3696,372,"""C:\Program Files\WinRAR\WinRAR.exe"" ""C:\Users...",C:\Program Files\WinRAR\WinRAR.exe,Probably manual launch or APC inject,439,1165,120,1,0,0,0,0,0,NKN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,11,0,0,0,0,0,1,0,NaN,2
229533,879a482d-c055-41fa-9e58-763908ee484b,1534329823794,1704,1008,"""C:\Users\admin\AppData\Roaming\doudoux.exe"" 2...",C:\Users\admin\AppData\Roaming\doudoux.exe,Child process,0,92,46,0,0,0,0,0,0,NKN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,3
13777,e07aed5f-191f-465f-9afe-59f0376ed297,1525198345726,3936,2648,"""C:\Windows\System32\cmd.exe"" /c PowerShell ""'...",C:\Windows\System32\cmd.exe,Child process,6,129,26,0,0,0,0,0,0,SCR-CMD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,3


In [ ]:
# process_data = process_data.drop(['task', 'CreationTimestamp', 'ProcessID', 'ParentPID', 'CommandLine', 'Image'] , axis = 1)
process_data["Scores_Network"] = process_data["Scores_Network"].astype(int)
process_data["Autostart"] = process_data["Autostart"].astype(int)
process_data["LowAccess"] = process_data["LowAccess"].astype(int)
process_data["ProcessType"] = process_data["ProcessType"].astype("category").cat.codes + 1
process_data["FileType"] = process_data["FileType"].astype("category").cat.codes + 1
process_data = process_data.drop(["Unnamed: 45"], axis=1)

In [ ]:
process_data.loc[process_data['label'] == 0, 'label'] = 3

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

task_data = task_data.sample(frac=1).reset_index(drop=True)
X_task_train, X_task_test, Y_task_train, Y_task_test = train_test_split(task_data["task"], task_data["label"],test_size=0.2)

list_of_task_train = X_task_train.tolist()
list_of_task_test = X_task_test.tolist()

process_train = process_data[process_data['task'].isin(list_of_task_train)]
process_test = process_data[process_data['task'].isin(list_of_task_test)]
# process_train = process_train.sample(frac=1).reset_index(drop=True)
process_test = process_test.sort_values(by='task')

Y_process_train = process_train['label']
X_process_train = process_train.drop(['label', 'task', 'CreationTimestamp', 'ProcessID', 'ParentPID', 'CommandLine', 'Image'], axis = 1)

Y_process_test= process_test['label']
X_process_test = process_test.drop(['label', 'task', 'CreationTimestamp', 'ProcessID', 'ParentPID', 'CommandLine', 'Image'], axis = 1)



In [ ]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_process_train,Y_process_train)
y_process_pred = model.predict(X_process_test)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix,f1_score,precision_score,recall_score
print(classification_report(Y_process_test,y_process_pred))
print(accuracy_score(Y_process_test,y_process_pred))
print(confusion_matrix(Y_process_test,y_process_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96     67783
           1       0.92      0.90      0.91     11290
           2       0.88      0.87      0.87     22003
           3       0.93      0.94      0.94     51231

    accuracy                           0.93    152307
   macro avg       0.92      0.92      0.92    152307
weighted avg       0.93      0.93      0.93    152307

0.9342512162934073
[[64884   255  1156  1488]
 [  330 10194   189   577]
 [ 1247   274 19202  1280]
 [ 1497   352  1369 48013]]


In [ ]:
# process_test["label"] = y_process_pred.tolist()
process_test = process_test.assign(label=y_process_pred.tolist()) 
# print(process_test)

In [ ]:
mal_tasks = list(set(process_test[process_test["label"] == 3]["task"].to_list()))
susp_tasks = []
for t in list(set(process_test[process_test["label"] == 2]["task"].to_list())):
  if t not in mal_tasks:
    susp_tasks.append(t)
nor_tasks = []
for t in list(set(process_test["task"].to_list())):
  if (t not in mal_tasks) and (t not in susp_tasks):
    nor_tasks.append(t)


In [ ]:
mal = pd.DataFrame({"task":mal_tasks, "label":  [3 for i in range(len(mal_tasks))]}, columns= ["task", "label"]) 
susp = pd.DataFrame({"task":susp_tasks, "label":  [2 for i in range(len(susp_tasks))]}, columns= ["task", "label"]) 
nor = pd.DataFrame({"task":nor_tasks, "label":  [1 for i in range(len(nor_tasks))]}, columns= ["task", "label"]) 

task_pred = pd.concat([mal, susp], sort=True)
task_pred = pd.concat([task_pred, nor], sort=True)
task_pred = task_pred.sort_values(by='task')

task_test = pd.DataFrame({"task":X_task_test.to_list(), "label":  Y_task_test.to_list()}, columns= ["task", "label"]) 
task_test = task_test.sort_values(by='task')


In [ ]:
print(classification_report(task_test["label"].tolist(),task_pred["label"].tolist()))
print(accuracy_score(task_test["label"].tolist(),task_pred["label"].tolist()))
print(confusion_matrix(task_test["label"].tolist(),task_pred["label"].tolist()))

              precision    recall  f1-score   support

           1       0.80      0.53      0.64      1510
           2       0.25      0.42      0.31       671
           3       0.94      0.95      0.95      8016

    accuracy                           0.85     10197
   macro avg       0.66      0.63      0.63     10197
weighted avg       0.88      0.85      0.86     10197

0.8539766598019025
[[ 801  572  137]
 [  78  280  313]
 [ 127  262 7627]]
